In [ ]:
pip install keras_resnet

  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=f8c8b72cd1cd2410e8dc93242bc3257fefd6ce782d2c565bc7cd6c324b06b629
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-resnet


In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input, Model
import tensorflow.keras as keras
import tensorflow as tf
from keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import History, ModelCheckpoint, TensorBoard
import numpy as np
import keras_resnet.blocks
#import tcn
#from tcn import TCN, tcn_full_summary
import pickle
from urllib.request import urlopen
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
#from tcn import compiled_tcn
import os
import keras_resnet.models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Wed Dec  2 20:32:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Utils

def create_new_model():

  shape, classes = (341, 513, 2), 10

  x = keras.layers.Input(shape)
  blocks = [2, 2, 2, 2]

  block = keras_resnet.blocks.basic_2d

  model = keras_resnet.models.ResNet2D(x, blocks=blocks, block=block, classes=classes)

  model.compile("adam", "categorical_crossentropy", ["accuracy"])

  return model




def load_model(depth, filters, kernel_size, save_dir="./"):
  model = create_new_model(depth, filters, kernel_size)
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  # model.load here
  if os.path.isfile(saved_model):
    model.load_weights(saved_model)
  else:
    print("NO MODEL ON DISK, USING BRAND NEW ONE")
  return model

def create_callbacks(save_dir="./"):
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  saved_hist  = save_dir + 'wavenet_classifier_training_history.csv'
  checkpointer = ModelCheckpoint(filepath=saved_model, monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
  #tensorboard  = TensorBoard(log_dir=save_dir)
  history_cb = tf.keras.callbacks.CSVLogger(saved_hist, separator=",", append=True)

  return [checkpointer, history_cb] # add tensorboard if needed

In [ ]:
all_X = []
all_Y = []
fulldatasetpath = './'
metadata = pd.read_csv('http://76.213.149.126:8081/metadata/UrbanSound8K.csv')
print(metadata)
for index, row in metadata.iterrows():
    
    file_name = 'fold'+str(row["fold"])+'/' + str(row["slice_file_name"])
    all_X.append(file_name.replace(".wav", "_x2.pkl"))
    all_Y.append(file_name.replace(".wav", "_y2.pkl"))

import requests
from multiprocessing.pool import ThreadPool
 
def download_url(url):
  #print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  path = url.split("/")
  path = path[-2] + "/" + path[-1]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    if not os.path.exists(os.path.dirname(path)):
      try:
          os.makedirs(os.path.dirname(path))
      except OSError as exc: # Guard against race condition
          if exc.errno != errno.EEXIST:
              raise
    with open(path, 'wb') as f:
      for data in r:
        f.write(data)
  return url

def set_shapes(x, y):
    y.set_shape([4, 10])
    x.set_shape([4, 341, 513, 2])
    return x, y

x_train = []
x_test = []
y_train = []
y_test = []

for x in all_X:
  if x[0:5] != "fold5":
    x_train.append(x)
    x_train.append(x.replace("_x2", "_x_augmented"))
  else:
    x_test.append(x)


for y in all_Y:
  if y[0:5] != "fold5":
    y_train.append(y)
    y_train.append(y)
  else:
    y_test.append(y)
    

def file_to_arr(t: tf.Tensor):
  x = []
  y = []

  urls = []

  for a in t:
    a = np.array(a)
    # check whether we have them
    # if not, schedule download
    if not os.path.exists(a[0]):
      urls.append("http://76.213.149.126:8081/audio/" + a[0].decode('UTF-8'))
    if not os.path.exists(a[1]):
      urls.append("http://76.213.149.126:8081/audio/" + a[1].decode('UTF-8'))
  
  if (len(urls) > 0):
    ThreadPool(4).map(download_url, urls)

  for a in t:
    a = np.array(a)
    #print(a[0])
    try:
      x.append(pickle.load(open(a[0], 'rb')))
      y.append(pickle.load(open(a[1], 'rb')))
    except:
      os.remove(a[0])
      os.remove(a[1])
      return file_to_arr(t)

  x = np.array(x)
  x = np.swapaxes(x, 1, 2)
  y = np.array(y).astype(int)
  res = (x,y)
  #print (res)
  return (res)

print((list(zip(x_train, y_train))))
train_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_train, y_train)))
train_ds = train_ds.shuffle(len(train_ds),seed=42)
train_ds = train_ds.batch(4)
train_ds = train_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
train_ds = train_ds.map(lambda x, y: set_shapes(x,y) )

print(train_ds.element_spec)
train_ds = train_ds.prefetch(32)

test_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_test, y_test)))
test_ds = test_ds.batch(4)
test_ds = test_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
test_ds = test_ds.prefetch(32)

In [ ]:
model = create_new_model()

In [ ]:
callbacks = create_callbacks("./drive/My Drive/wavenet-test/resnet/2_2_2_2_default")

model.summary()

Model: "res_net2d"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 341, 513, 2) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 171, 257, 64) 6272        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 171, 257, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (Activation)         (None, 171, 257, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________

In [ ]:
!mkdir "./drive/My Drive/wavenet-test/resnet"

In [ ]:
model.fit(train_ds, validation_data=test_ds, callbacks=callbacks, epochs=300)

Epoch 1/300
3898/3898 [==============================] - 712s 183ms/step - loss: 2.4161 - accuracy: 0.1845 - val_loss: 2.0877 - val_accuracy: 0.3130
Epoch 2/300
3898/3898 [==============================] - 631s 162ms/step - loss: 2.0215 - accuracy: 0.3032 - val_loss: 1.8558 - val_accuracy: 0.2906
Epoch 3/300
3898/3898 [==============================] - 632s 162ms/step - loss: 1.7542 - accuracy: 0.4071 - val_loss: 1.7891 - val_accuracy: 0.4338
Epoch 4/300
3898/3898 [==============================] - 632s 162ms/step - loss: 1.5708 - accuracy: 0.4785 - val_loss: 1.6842 - val_accuracy: 0.4754
Epoch 5/300
3898/3898 [==============================] - 631s 162ms/step - loss: 1.4495 - accuracy: 0.5266 - val_loss: 1.5661 - val_accuracy: 0.4915
Epoch 6/300
3898/3898 [==============================] - 630s 162ms/step - loss: 1.3295 - accuracy: 0.5730 - val_loss: 1.4735 - val_accuracy: 0.5609
Epoch 7/300
3898/3898 [==============================] - 632s 162ms/step - loss: 1.2439 - accuracy: 0.6058